In [20]:
import glob, os, json

import pandas as pd
import numpy as np

import plotly.express as px

pd.set_option('display.max_columns', 35)
pd.set_option('display.max_colwidth', None)


import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.neural_network import MLPRegressor
import sklearn.metrics as metrics

In [21]:
# https://towardsdatascience.com/how-to-convert-json-into-a-pandas-dataframe-100b2ae1e0d8

json_dir = os.getcwd() +'/data/'

json_pattern = os.path.join(json_dir, '*.json')
file_list = glob.glob(json_pattern)

dfs = []

for file in file_list:
    with open(file) as f:
        json_data = pd.json_normalize(json.loads(f.read()))
    dfs.append(json_data)

df = pd.concat(dfs)

In [22]:
# convert to fees to Algo, tx amount to more widely known USDC format 
df.fee = df.fee / 1000000
df['asset-transfer-transaction.amount'] = df['asset-transfer-transaction.amount'] / 1000000

## EDA

In [23]:
df.shape

(66855, 32)

In [24]:
# Number of Unique Rounds in the data
df['confirmed-round'].nunique()

62247

In [25]:
df.head(5)

,close-rewards,closing-amount,confirmed-round,fee,first-valid,genesis-hash,id,intra-round-offset,last-valid,lease,receiver-rewards,round-time,sender,sender-rewards,tx-type,asset-transfer-transaction.amount,asset-transfer-transaction.asset-id,asset-transfer-transaction.close-amount,asset-transfer-transaction.receiver,signature.sig,signature.logicsig.args,signature.logicsig.logic,signature.logicsig.multisig-signature.subsignature,signature.logicsig.multisig-signature.threshold,signature.logicsig.multisig-signature.version,genesis-id,group,note,signature.multisig.subsignature,signature.multisig.threshold,signature.multisig.version,asset-transfer-transaction.close-to
0,0,0,12011008,0.001,12011004,wGHE2Pwdvd7S12BL5FaOP20EGYesN73ktiC1qzkkit8=,RVXQYH7F3674P635CRUSCBEKXIJ7AVZKR3A6E6L6UN6FSFH7Y6EA,3,12012004,xynVQU2YOnPysyrADpmZOYxRQqdRfSXyRpB4iHIguZo=,0,1613100285,ZG54ZBZ5LVWV3MTGOPDSKCBL5LEQTAPUTN5OQQZUMTAYV3JIICA7G3RJZE,1024,axfer,8830.58,31566704,0,2UEQTE5QDNXPI7M3TU44G6SYKLFWLPQO7EBZM7K7MHMQQMFI4QJPLHQFHM,iMkZ8FLAk4BNMow/XKbSm5l8elxaMKMEeWvvY5Da1vb/W3b/8+NteORE6t0jKjB6lo+yvtCSUECmSZEG4zjEAw==,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,12011035,0.001,12011032,wGHE2Pwdvd7S12BL5FaOP20EGYesN73ktiC1qzkkit8=,TG74PEHX5473K5AF5QXKWVYIQKCURBO6NKVMQ6EHNM7VDF6EVXSQ,21,12012032,O66cq4LsYu5/Kg5bD/PE/vgtaiEHKLxddXJcjW6mUtg=,0,1613100403,ZG54ZBZ5LVWV3MTGOPDSKCBL5LEQTAPUTN5OQQZUMTAYV3JIICA7G3RJZE,0,axfer,53.68,31566704,0,2UEQTE5QDNXPI7M3TU44G6SYKLFWLPQO7EBZM7K7MHMQQMFI4QJPLHQFHM,EqM5NA5iUiZIhDaoJOhXYP4LyYZsadnZmg8PsYHM5nn1r+bDB579HhWHPIt6mfEjeJ+iZpYWfz5rq31GiTmaDw==,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,12011103,0.001,12011100,wGHE2Pwdvd7S12BL5FaOP20EGYesN73ktiC1qzkkit8=,3Y3THGYKCZYCHOPBFTCDQFOJNESHOEITLW23ZKNKDG7VZL4OAR3A,7,12012100,8iwPEL6THRZ2LcNxlWQtGtaGhpkAAJKYNSZyChZ11T0=,0,1613100703,ZG54ZBZ5LVWV3MTGOPDSKCBL5LEQTAPUTN5OQQZUMTAYV3JIICA7G3RJZE,0,axfer,271.20,31566704,0,2UEQTE5QDNXPI7M3TU44G6SYKLFWLPQO7EBZM7K7MHMQQMFI4QJPLHQFHM,kd96Zp+8J021EKXSvkECY1jinMYV/f7E/PBu1V4BoSKowE8r+t7312q0s9BzbmTL0JlhUrAijIirAfiwp+tcDg==,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,12011144,0.001,12011140,wGHE2Pwdvd7S12BL5FaOP20EGYesN73ktiC1qzkkit8=,FXI34SEYYXZCQPBBYI6KV57AW4SBEWGODWOL2BZTROITT4I4K6ZQ,9,12012140,0xTlqyl8gOG2ZA2cnns8eyPg4vMGAE2QOg5hKteoGc4=,0,1613100885,ZG54ZBZ5LVWV3MTGOPDSKCBL5LEQTAPUTN5OQQZUMTAYV3JIICA7G3RJZE,1024,axfer,41.71,31566704,0,2UEQTE5QDNXPI7M3TU44G6SYKLFWLPQO7EBZM7K7MHMQQMFI4QJPLHQFHM,bfWnsZgOaUpzJkM16XdQsjhvq7YrHRubiRv0dBrrffsi14qtfRh0h52dpk9dj70vukb0FEFlAme9/o64k5XDDQ==,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,12011171,0.001,12011167,wGHE2Pwdvd7S12BL5FaOP20EGYesN73ktiC1qzkkit8=,TUOGQVNTBBJVEG2UH43TSCA5LFZ777A7KNDSHJ7HDZB2D54QCTLA,9,12012167,gIa/Q3I76LoXOqyhUTfgAd71L19Wta6/M3s0Acc4Mdc=,0,1613101005,ZG54ZBZ5LVWV3MTGOPDSKCBL5LEQTAPUTN5OQQZUMTAYV3JIICA7G3RJZE,0,axfer,260.38,31566704,0,2UEQTE5QDNXPI7M3TU44G6SYKLFWLPQO7EBZM7K7MHMQQMFI4QJPLHQFHM,KDwXNas9Dij20+mPliC6q2SIuQ1FLb+sxA0GloZJDVTte0sK28612eiGwtaGn7WC+wvsve5kFH5USVaIdZFUBw==,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df.columns

Index(['close-rewards', 'closing-amount', 'confirmed-round', 'fee',
       'first-valid', 'genesis-hash', 'id', 'intra-round-offset', 'last-valid',
       'lease', 'receiver-rewards', 'round-time', 'sender', 'sender-rewards',
       'tx-type', 'asset-transfer-transaction.amount',
       'asset-transfer-transaction.asset-id',
       'asset-transfer-transaction.close-amount',
       'asset-transfer-transaction.receiver', 'signature.sig',
       'signature.logicsig.args', 'signature.logicsig.logic',
       'signature.logicsig.multisig-signature.subsignature',
       'signature.logicsig.multisig-signature.threshold',
       'signature.logicsig.multisig-signature.version', 'genesis-id', 'group',
       'note', 'signature.multisig.subsignature',
       'signature.multisig.threshold', 'signature.multisig.version',
       'asset-transfer-transaction.close-to'],
      dtype='object')

In [27]:
df['round-time'] = pd.to_datetime(df['round-time'], unit='s')

In [28]:
ag_df = df.groupby(by=[df['round-time'].dt.date])['asset-transfer-transaction.amount'].agg(volume='sum', mean='mean')
ag_df

,volume,mean
round-time,,
2021-01-22,1.927405e+07,275343.513714
2021-01-23,2.036411e+07,452535.767778
2021-01-24,1.661494e+06,75522.470000
2021-01-25,1.348840e+07,85369.613481
2021-01-26,9.178005e+06,31217.703163
...,...,...
2021-10-05,6.982135e+07,347369.885456
2021-10-06,2.839528e+07,155165.482715
2021-10-07,3.889015e+07,18733.212731


##  Feature Engineering

In [30]:
# inserting new column with yesterday's values
ag_df.loc[:,'volume-1'] = ag_df.loc[:,'volume'].shift()

# inserting another column with difference between yesterday and day before yesterday's consumption values.
ag_df.loc[:,'volume_diff'] = ag_df.loc[:,'volume'].diff()

ag_df

,volume,mean,volume-1,volume_diff
round-time,,,,
2021-01-22,1.927405e+07,275343.513714,NaN,NaN
2021-01-23,2.036411e+07,452535.767778,1.927405e+07,1.090064e+06
2021-01-24,1.661494e+06,75522.470000,2.036411e+07,-1.870262e+07
2021-01-25,1.348840e+07,85369.613481,1.661494e+06,1.182690e+07
2021-01-26,9.178005e+06,31217.703163,1.348840e+07,-4.310394e+06
...,...,...,...,...
2021-10-05,6.982135e+07,347369.885456,1.433945e+07,5.548189e+07
2021-10-06,2.839528e+07,155165.482715,6.982135e+07,-4.142606e+07
2021-10-07,3.889015e+07,18733.212731,2.839528e+07,1.049487e+07


In [31]:
# dropping NAs
ag_df = ag_df.dropna()

### Defining Training and Testing Data

In [32]:
train = ag_df[:int(len(ag_df)*0.8)]
test = ag_df[int(len(ag_df)*0.8):]

X_train, X_test, y_train, y_test = train.drop('volume', axis=1), test.drop('volume', axis=1), train['volume'], test['volume']

## Trainining Algorithm

In [33]:
model = RandomForestRegressor(random_state=42)
param_search = { 
    'n_estimators': [10, 20, 50, 100],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [i for i in range(1,15)]
}
tscv = TimeSeriesSplit(n_splits=4)
gsearch = GridSearchCV(estimator=model, cv=tscv, param_grid=param_search, scoring = 'neg_mean_squared_error')
gsearch.fit(X_train, y_train)
rf_best_score = gsearch.best_score_
rf_best_model = gsearch.best_estimator_
print(f"{rf_best_model} at {rf_best_score}")

RandomForestRegressor(max_depth=14, max_features='auto', n_estimators=10,
                      random_state=42) at -117401377358877.33


In [34]:
model = MLPRegressor(random_state=1)
param_search = { 
    'max_iter':[ 100, 200, 400, 600, 800, 1000],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'activation':['identity', 'logistic', 'tanh', 'relu'],
}

tscv = TimeSeriesSplit(n_splits=4)
gsearch = GridSearchCV(estimator=model, cv=tscv, param_grid=param_search, scoring = 'neg_mean_squared_error')
gsearch.fit(X_train, y_train)
best_score = gsearch.best_score_
best_model = gsearch.best_estimator_
print(f"{best_model} at {best_score}")

MLPRegressor(activation='identity', max_iter=100, random_state=1,
             solver='lbfgs') at -0.013668465516815352


## Scoring

In [35]:
def regression_results(y_true, y_pred):
    # Regression metrics
    explained_variance = metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error = metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)
    
    print('explained_variance: ', round(explained_variance, 4))    
    print('mean_squared_log_error: ', round(mean_squared_log_error, 4))
    print('r2: ', round(r2, 4))
    print('MAE: ', round(mean_absolute_error, 4))
    print('MSE: ', round(mse, 4))
    print('RMSE: ', round(np.sqrt(mse), 4))

In [36]:
y_true = y_test.values
y_pred = best_model.predict(X_test)
regression_results(y_true, y_pred)

explained_variance:  1.0
mean_squared_log_error:  0.0
r2:  1.0
MAE:  0.1111
MSE:  0.0186
RMSE:  0.1363


In [37]:
y_true = y_test.values
y_pred = rf_best_model.predict(X_test)
regression_results(y_true, y_pred)

explained_variance:  0.8246
mean_squared_log_error:  0.4097
r2:  0.8122
MAE:  7170569.465
MSE:  317721498450274.56
RMSE:  17824743.994
